In [7]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
import gc
import time
import sys
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

from catboost import Pool, CatBoostClassifier



In [6]:
#CatBoost

train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

idx = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

df_list = [train_df, test_df]
new_df_list = []
new_train_df = pd.DataFrame()
new_train_df['target'] = train_df['target']
new_train_df['ID_code'] = train_df['ID_code']
new_test_df = pd.DataFrame()
new_test_df['ID_code'] = test_df['ID_code']
new_df_list = [new_train_df, new_test_df]
for i in range(0, len(df_list)):        
    for feat in idx:
        mean = df_list[i].loc[:,feat].mean()
        new_df_list[i][feat] = df_list[i][feat]
#         new_df_list[i]["mm_" + feat] = np.round(df_list[i][feat] - mean , 2) 
#         if mean > 0:
#             new_df_list[i]["is_more_" + feat] = np.where( df_list[i][feat] > mean, 10, -10 )
#         if mean < 0: 
#             new_df_list[i]["is_more_" + feat] = np.where( df_list[i][feat] < mean, 10, -10 )
            
features = [c for c in new_train_df.columns if c not in ['ID_code', 'target']]
target = new_train_df['target']

N_FOLDS = 5.0

model = CatBoostClassifier(loss_function="Logloss", eval_metric="AUC", iterations=2000, early_stopping_rounds= 500, l2_leaf_reg= 9, learning_rate= 0.1)
#model = CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 1500, l2_leaf_reg= 9, learning_rate= 0.15)


kf = KFold(n_splits=int(N_FOLDS), random_state=2019, shuffle=True)

y_valid_pred = 0 * target
y_test_pred = 0
y_train_pred = 0 #storing results for ensemble

new_train_df.info()

for idx, (train_index, valid_index) in enumerate(kf.split(new_train_df)):
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_train, X_valid = new_train_df[features].iloc[train_index,:], new_train_df[features].iloc[valid_index,:]
    _train = Pool(X_train, label=y_train)
    _valid = Pool(X_valid, label=y_valid)
    print( "\nFold ", idx)
    fit_model = model.fit(_train,
                          eval_set=_valid,
                          use_best_model=True,
                          verbose=200
                         )
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  auc = ", roc_auc_score(y_valid, pred) )
    y_valid_pred.iloc[valid_index] = pred
    y_test_pred += fit_model.predict_proba(new_test_df[features])[:,1]
    y_train_pred += fit_model.predict_proba(new_train_df[features])[:,1]
y_test_pred /= N_FOLDS
y_train_pred /= N_FOLDS

#save an intermediate submission
sub_df1 = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df1["target"] = y_test_pred
sub_df1.to_csv("submission_catboost.csv", index=False)

ens_cat_df = pd.DataFrame({"ID_code":train_df["ID_code"].values})
ens_cat_df["output"] = y_train_pred
ens_cat_df.to_csv("../input/catboost_output.csv")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, target to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB

Fold  0
0:	test: 0.5364710	best: 0.5364710 (0)	total: 130ms	remaining: 4m 20s
200:	test: 0.8776868	best: 0.8776868 (200)	total: 22.6s	remaining: 3m 21s
400:	test: 0.8900528	best: 0.8900528 (400)	total: 45.2s	remaining: 3m
600:	test: 0.8947133	best: 0.8947133 (600)	total: 1m 7s	remaining: 2m 36s
800:	test: 0.8963380	best: 0.8963380 (800)	total: 1m 29s	remaining: 2m 13s
1000:	test: 0.8966581	best: 0.8967353 (993)	total: 1m 51s	remaining: 1m 51s
1200:	test: 0.8970510	best: 0.8970800 (1194)	total: 2m 13s	remaining: 1m 29s
1400:	test: 0.8970907	best: 0.8972818 (1294)	total: 2m 35s	remaining: 1m 6s
1600:	test: 0.8970987	best: 0.8972818 (1294)	total: 2m 57s	remaining: 44.2s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8972817673
bestIteration = 1294

Shrink model to first 1295 iterations

In [11]:
#####LightGBM
param = {
            'num_leaves': 15, #was 10
            'max_bin': 119,
            'min_data_in_leaf': 11,
            'learning_rate': 0.01,
            'min_sum_hessian_in_leaf': 0.00245,
            'bagging_fraction': 1.0, 
            'bagging_freq': 5, 
            'feature_fraction': 0.05,
            'lambda_l1': 4.972,
            'lambda_l2': 30.0,    #2.276,
            'min_gain_to_split': 0.65,
            'max_depth': 17, #was 14
            'save_binary': True,
            'seed': 1337,
            'feature_fraction_seed': 1337,
            'bagging_seed': 1337,
            'drop_seed': 1337,
            'data_random_seed': 1337,
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'verbose': 1,
            'metric': 'auc',
            'is_unbalance': True,
            'boost_from_average': True, #was false
        }
num_round = 15000
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
train_preds = np.zeros(len(train_df))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
    train_preds += clf.predict(train_df[features], num_iteration=clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub_lgb = pd.DataFrame({"ID_code": train_df.ID_code.values})
sub["target"] = predictions
sub_lgb["output"] = train_preds
sub.to_csv("submission.csv", index=False)
sub_lgb.to_csv("../input/submission_lgbm.csv")

Fold 0
Training until validation scores don't improve for 200 rounds.
[1000]	training's auc: 0.906267	valid_1's auc: 0.885823
[2000]	training's auc: 0.917022	valid_1's auc: 0.891728
[3000]	training's auc: 0.925637	valid_1's auc: 0.895555
[4000]	training's auc: 0.932462	valid_1's auc: 0.897559
[5000]	training's auc: 0.938223	valid_1's auc: 0.898776
[6000]	training's auc: 0.943289	valid_1's auc: 0.899484
[7000]	training's auc: 0.947816	valid_1's auc: 0.899977
Early stopping, best iteration is:
[7590]	training's auc: 0.950327	valid_1's auc: 0.900103
Fold 1
Training until validation scores don't improve for 200 rounds.
[1000]	training's auc: 0.906621	valid_1's auc: 0.881273
[2000]	training's auc: 0.917322	valid_1's auc: 0.887924
[3000]	training's auc: 0.92588	valid_1's auc: 0.891659
[4000]	training's auc: 0.932694	valid_1's auc: 0.89421
[5000]	training's auc: 0.938411	valid_1's auc: 0.895752
[6000]	training's auc: 0.943499	valid_1's auc: 0.89674
[7000]	training's auc: 0.948001	valid_1's au

In [ ]:
##NN Ensemble
from keras import models
from kear import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape(200000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
             loss = 'binary_crossentropy',
             metrics = ['ac'])